# 3D line fitting + computed vector direction choice
In this notebook I will present a way of imporving 3D line fitting by using time values. Using this way I improved my LB score from `1.26` to `1.193`. With chosing a proper direciton of predicted vector the score could be around `0.8`.

In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import math
from tqdm import tqdm
from random import sample

In [2]:
subset = 'train'
limit = 10_000

dataset_path = Path(r'/kaggle/input/icecube-neutrinos-in-deep-ice')
test_meta_data = pd.read_parquet(dataset_path / f'{subset}_meta.parquet')
if limit is not None:
    test_meta_data = test_meta_data[:limit]
sensor_geo = pd.read_csv(dataset_path / 'sensor_geometry.csv')

In [3]:
def angular_dist_score(az_true, zen_true, az_pred, zen_pred):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))

In [4]:
def _cartesian_to_spherical(x, y, z):
    r = math.sqrt(x * x + y * y + z * z)
    c = -1 if y < 0 else 1
    azimuth = math.acos(x / math.sqrt(x * x + y * y)) * c
    zenith = math.acos(z / r)
    return azimuth, zenith

def _adjust_spherical(azimuth, zenith):
    if azimuth < 0 and zenith < 0:
        azimuth *= -1
        zenith *= -1
    if azimuth < 0:
        azimuth += math.pi * 2
    elif zenith < 0:
        zenith += math.pi
    return azimuth, zenith

def cartesian_to_spherical(pred_direction: np.ndarray):
    try:
        a, z = _adjust_spherical(*_cartesian_to_spherical(*(pred_direction.tolist())))
        if np.isnan(a):
            a = 0.0
        if np.isnan(z):
            z = 0.0
        return a, z
    except:
        return 0.0, 0.0

In [5]:
total_score = {'svd': 0.0, 'svd+t': 0.0, 'best': 0.0}

perc = 0.25
prev_batch_id = -1

progress_bar = tqdm(range(len(test_meta_data)))
for i in progress_bar:
    meta_data_row = test_meta_data.iloc[i]
    batch_id = int(meta_data_row['batch_id'])
    first_pulse_index = int(meta_data_row['first_pulse_index'])
    last_pulse_index = int(meta_data_row['last_pulse_index'])
    
    if batch_id != prev_batch_id:
        train_batch = pd.read_parquet(dataset_path / (f'{subset}/batch_{batch_id}.parquet'))
        prev_batch_id = batch_id

    event_data = train_batch.iloc[first_pulse_index:last_pulse_index + 1]
    event_data = event_data[~event_data['auxiliary']]
    if len(event_data) > 800:
        event_data = event_data.sort_values('charge', ascending=False)[:800]

    event_data_with_pos = pd.merge(event_data, sensor_geo, on='sensor_id', how='left')

    time_sorted = event_data_with_pos.sort_values('time', ascending=True)
    m = max(1, int(perc*len(time_sorted)))
    first = time_sorted.iloc[:m]
    last = time_sorted.iloc[-m:]
    
    first = np.array([first['x'].mean(), first['y'].mean(), first['z'].mean()])
    last = np.array([last['x'].mean(), last['y'].mean(), last['z'].mean()])

    time_dir = last - first
    time_dir /= np.linalg.norm(time_dir)

    data = np.concatenate(
        (np.array(event_data_with_pos['x'])[:, np.newaxis], 
         np.array(event_data_with_pos['y'])[:, np.newaxis], 
         np.array(event_data_with_pos['z'])[:, np.newaxis]), 
        axis=1
    )

    _, _, v = np.linalg.svd(data - data.mean(axis=0))

    pred_direction = -v[0]
    pred_direction_neg = -pred_direction.copy()
    
    # check if predicted direction is aligned with vector computed based on time
    if np.dot(time_dir, pred_direction) > 0:
        pred_direction_t = -pred_direction.copy()
    else:
        pred_direction_t = pred_direction.copy()
    
    azimuth, zenith = cartesian_to_spherical(pred_direction)
    azimuth_t, zenith_t = cartesian_to_spherical(pred_direction_t)
    azimuth_neg, zenith_neg = cartesian_to_spherical(pred_direction_neg)
    
    if subset == 'train':
        true_azimuth = meta_data_row['azimuth']
        true_zenith = meta_data_row['zenith']
        
        score = angular_dist_score(
            true_azimuth, true_zenith,
            azimuth, zenith
        )
        score_adj = angular_dist_score(
            true_azimuth, true_zenith,
            azimuth_t, zenith_t
        )
        score_neg = angular_dist_score(
            true_azimuth, true_zenith,
            azimuth_neg, zenith_neg
        )
        total_score['svd'] += score
        total_score['svd+t'] += score_adj
        total_score['best'] += min(score, score_neg)
    
        progress_bar.set_description(
            f'svd: {total_score["svd"]/(i + 1):.3f}, '
            f'svd+t: {total_score["svd+t"]/(i + 1):.3f}, '
            f'best: {total_score["best"]/(i + 1):.3f} | '
        )
total_score['svd'] /= len(test_meta_data)
total_score['svd+t'] /= len(test_meta_data)
total_score['best'] /= len(test_meta_data)

svd: 1.248, svd+t: 1.114, best: 0.765 | :   1%|          | 68/10000 [00:07<08:12, 20.17it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide
svd: 1.287, svd+t: 1.214, best: 0.830 | : 100%|██████████| 10000/10000 [02:05<00:00, 79.57it/s]


In [6]:
result = pd.DataFrame(
    data=total_score.values(),
    columns=['score'],
    index=total_score.keys()
)
result.index.name = 'method'

In [7]:
display(result)

,score
method,
svd,1.286676
svd+t,1.214241
best,0.829636
